In [3]:
import pandas as pd
runways = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/runways.csv')
regions = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/regions.csv')
navaids = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/navaids.csv')
countries = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/countries.csv')
airports = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airports.csv')
airport_frequencies = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airport-frequencies.csv')
airport_comments = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airport-comments.csv')
airports

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,506791,00AN,small_airport,Katmai Lodge Airport,59.093287,-156.456699,80.0,NaN,US,US-AK,King Salmon,no,00AN,NaN,00AN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75047,46378,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,40.0,EU,GB,GB-ENG,Sealand,no,NaN,NaN,NaN,http://www.sealandgov.org/,https://en.wikipedia.org/wiki/Principality_of_...,Roughs Tower Helipad
75048,307326,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,11.0,AF,TF,TF-U-A,Grande Glorieuse,no,NaN,NaN,NaN,NaN,NaN,NaN
75049,346788,ZZ-0003,small_airport,Fainting Goat Airport,32.110587,-97.356312,690.0,NaN,US,US-TX,Blum,no,87TX,NaN,87TX,NaN,NaN,NaN
75050,342102,ZZZW,closed,Scandium City Heliport,69.355287,-138.939310,4.0,NaN,CA,CA-YT,(Old) Scandium City,no,NaN,NaN,NaN,NaN,NaN,"ZZZW, ZZZW, ZYW, YK96"


## 1. How many airports, airfields and heliports exist in each country and continent?

### airports and heliports in each country, each airport has its own airfield. 

In [2]:
import country_converter as coco
wf = airports[airports["type"].str.contains("closed") == False]
wf = wf[wf["type"].str.contains("seaplane_base") == False]
wf = wf.replace(to_replace=["large_airport", "medium_airport",'small_airport'],value="airport")
wf = wf.groupby('type')['iso_country'].apply(lambda x: x.value_counts()).reset_index()
wf = wf.rename(columns = {'level_1':'country_code','iso_country':'count'})
wf['country_name'] = wf.country_code.apply(lambda x: coco.convert(names=x, to='name_short', not_found=None))
wf.sort_values('country_code')


,type,country_code,count,country_name
368,heliport,AD,2,Andorra
261,heliport,AE,200,United Arab Emirates
96,airport,AE,41,United Arab Emirates
319,heliport,AF,8,Afghanistan
70,airport,AF,67,Afghanistan
...,...,...,...,...
12,airport,ZA,571,South Africa
42,airport,ZM,117,Zambia
351,heliport,ZM,4,Zambia
340,heliport,ZW,5,Zimbabwe


### airports and heliports in each continent, each airport has its own airfield. Oceania is a continent?

In [3]:
import pycountry_convert as pc
wf = airports[airports["type"].str.contains("closed") == False]
wf = wf[wf["type"].str.contains("seaplane_base") == False]
wf = wf.replace(to_replace=["large_airport", "medium_airport",'small_airport','balloonport'],value="airport")
wf['continent'] = wf['continent'].fillna('NA')
wf = wf.groupby('type')['continent'].apply(lambda x: x.value_counts()).reset_index()
wf = wf.rename(columns = {'level_1':'continent_code','continent':'count'})
wf['continent_name'] = [pc.convert_continent_code_to_continent_name(x) for x in wf['continent_code']]
wf.sort_values('continent_code')



,type,continent_code,count,continent_name
4,airport,AF,3572,Africa
12,heliport,AF,209,Africa
6,airport,AN,30,Antarctica
13,heliport,AN,9,Antarctica
3,airport,AS,4023,Asia
8,heliport,AS,5712,Asia
2,airport,EU,6719,Europe
10,heliport,EU,1714,Europe
0,airport,NA,19420,North America
7,heliport,NA,8805,North America


## 2. What is the average elevation of the airports, airfields and heliports in each country?

In [4]:
import country_converter as coco
wf = airports[airports["type"].str.contains("closed") == False]
wf = wf[wf["type"].str.contains("seaplane_base") == False]
wf = wf.replace(to_replace=["large_airport", "medium_airport",'small_airport'],value="airport")
wf = wf.groupby([wf.iso_country, wf.type])['elevation_ft'].mean().round(2)
wf = pd.DataFrame(wf).reset_index()
wf = wf.rename(columns = {'iso_country':'country_code','elevation_ft':'average_elevation_ft'})
wf['country_name'] = wf.country_code.apply(lambda x: coco.convert(names=x, to='name_short', not_found=None))


wf

,country_code,type,average_elevation_ft,country_name
0,AD,heliport,3450.00,Andorra
1,AE,airport,124.97,United Arab Emirates
2,AE,heliport,113.79,United Arab Emirates
3,AF,airport,5028.52,Afghanistan
4,AF,heliport,3198.33,Afghanistan
...,...,...,...,...
404,ZA,heliport,3918.43,South Africa
405,ZM,airport,3570.41,Zambia
406,ZM,heliport,2658.50,Zambia
407,ZW,airport,3037.72,Zimbabwe


## 3. What is the estimated population of each country?

In [1]:
#Initializing spark session
from pyspark.sql import SparkSession

# create SparkSession
spark = SparkSession.builder.appName("ReadTextFile").getOrCreate()

Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m
23/06/01 14:43:29 WARN Utils: Your hostname, georgemicha-airportpipe-vz5635a85z0 resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface eth0)
23/06/01 14:43:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/01 14:43:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Using PySpark to find the estimated population of each country

In [6]:
from pyspark.sql.functions import col,sum,count
header_names = ["geoname_id", "name", "asciiname","alternatenames","latitude","longitude",
"feature_class","feature_code","country_code","altcountry_code","admin1_code","admin2_code"
,"admin3_code","admin4_code","population","elevation","DEM","timezone","modification_date" ]
path = "/workspace/Airport_Pipeline/Geonames_data/allCountries.txt"


#text_file = spark.read.format("csv").option("delimiter", "\t").load("./Geonames_data/allCountries.txt")
df = spark.read.option("header", "true").option("delimiter", "\t").csv(path).toDF(*header_names)
# show the first few lines


grouped = df.groupBy("country_code").agg(sum("population").alias("sum_population"), count("country_code").alias("count_country"))
# grouped by country_code and population summed also another column count_country to count how many times the country_code that was summed came up
grouped.show()
#grouped.filter(grouped.country_code == 'NG').select("country_code","sum_population","count_country").show()

+------------+--------------+-------------+
|country_code|sum_population|count_country|
+------------+--------------+-------------+
|          AZ|   3.1204388E7|        10162|
|          AM|     8600255.0|        31745|
|          AT|    5.135821E7|        52588|
|          AD|      235502.0|         3278|
|          BN|     1092427.0|          943|
|          BD|   4.9220236E8|        59135|
|          AU|  1.22647619E8|       214078|
|          BL|       14438.0|          132|
|          BM|      132590.0|          668|
|          BR|  9.85681984E8|       130898|
|          AW|      313943.0|          483|
|          BH|     5168513.0|          558|
|          AI|       46366.0|          233|
|          AQ|        4438.0|        18531|
|          AL|   1.1037641E7|         9406|
|          AE|   3.5616653E7|         6288|
|          AG|      298192.0|          775|
|          BA|   1.0279473E7|        51880|
|          BF|   6.3004738E7|        11634|
|          BG|   2.6348095E7|   

# 4. How many cities/towns/settlements in each country?

### All values in the municipality column are distinct in complete rows meaning all counting the values of each country code in the iso_country column can give us the above answer. using airports.csv

In [7]:

import country_converter as coco
airports['iso_country'] = airports['iso_country'].fillna('NA')
wf =airports.iso_country.value_counts()
wf = pd.DataFrame(wf).reset_index()
wf['country_name'] = wf.iso_country.apply(lambda x: coco.convert(names=x, to='name_short', not_found=None))
wf

#airports['iso_country'].isnull().sum()
#mask = airports['iso_country'].isna()
#rows_with_nans = airports.loc[mask, :]
#rows_with_nans


,iso_country,count,country_name
0,US,30003,United States
1,BR,6669,Brazil
2,JP,3312,Japan
3,CA,2959,Canada
4,AU,2553,Australia
...,...,...,...
240,GS,1,South Georgia and South Sandwich Is.
241,BL,1,St. Barths
242,GM,1,Gambia
243,YT,1,Mayotte


# 4. How many cities/towns/settlements in each country? 
#### using allCountries.txt

In [8]:
from pyspark.sql.functions import col,sum,count
header_names = ["geoname_id", "name", "asciiname","alternatenames","latitude","longitude",
"feature_class","feature_code","country_code","altcountry_code","admin1_code","admin2_code"
,"admin3_code","admin4_code","population","elevation","DEM","timezone","modification_date" ]
path = "/workspace/Airport_Pipeline/Geonames_data/allCountries.txt"


#text_file = spark.read.format("csv").option("delimiter", "\t").load("./Geonames_data/allCountries.txt")
df = spark.read.option("header", "true").option("delimiter", "\t").csv(path).toDF(*header_names)

#has_not_null = df.filter(col("country_code").isNotNull())
grouped = df.groupBy("country_code").count()
grouped.show()

+------------+------+
|country_code| count|
+------------+------+
|          AZ| 10162|
|          AM| 31745|
|          AT| 52588|
|          AD|  3278|
|          BN|   943|
|          BD| 59135|
|          AU|214078|
|          BL|   132|
|          BM|   668|
|          BR|130898|
|          AW|   483|
|          BH|   558|
|          AI|   233|
|          AQ| 18531|
|          AL|  9406|
|          AE|  6288|
|          AG|   775|
|          BA| 51880|
|          BF| 11634|
|          BG| 18206|
+------------+------+
only showing top 20 rows



# 5. What is the min, max and average elevation of the cities per country?

In [3]:
from pyspark.sql.functions import avg, min, max, col
header_names = ["geoname_id", "name", "asciiname","alternatenames","latitude","longitude",
"feature_class","feature_code","country_code","altcountry_code","admin1_code","admin2_code"
,"admin3_code","admin4_code","population","elevation","DEM","timezone","modification_date" ]
path = "/workspace/Airport_Pipeline/Geonames_data/allCountries.txt"


#text_file = spark.read.format("csv").option("delimiter", "\t").load("./Geonames_data/allCountries.txt")
df = spark.read.option("header", "true").option("delimiter", "\t").csv(path).toDF(*header_names)

result = df.groupBy("country_code") \
               .agg(min(col("elevation")).alias("min_elevation"),
                    max(col("elevation")).alias("max_elevation"),
                    avg(col("elevation")).alias("avg_elevation"))

result.show()

23/06/01 14:44:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: AD, 
 Schema: AD, _c15
Expected: _c15 but found: 
CSV file: file:///workspace/Airport_Pipeline/Geonames_data/allCountries.txt


+------------+-------------+-------------+------------------+
|country_code|min_elevation|max_elevation|     avg_elevation|
+------------+-------------+-------------+------------------+
|        null|        -1125|         8848|323.51162790697674|
|          AD|            0|         2943|           1690.44|
|          AE|            0|          954| 482.3766233766234|
|          AF|         1002|          998|2640.5309108069846|
|          AG|           18|          565|             337.5|
|          AI|           38|            7|              22.5|
|          AL|         1000|          991| 1416.888888888889|
|          AM|         1064|          890|            2338.7|
|          AN|         null|         null|              null|
|          AO|            0|          999|1080.6319343945972|
|          AQ|           -1|          997|1367.2830150753769|
|          AR|            0|           97|1701.2186379928316|
|          AS|           -1|           99|  69.7492291880781|
|       

# 6. Which are the highest and lowest elevated cities in the world with populations > 100000?

In [4]:
filtered_cities_df = df.filter(col("population") > 100000)

# Find the highest elevated city
highest_elevation_city = filtered_cities_df.orderBy(col("elevation").desc()).first()

# Find the lowest elevated city
lowest_elevation_city = filtered_cities_df.orderBy(col("elevation")).first()

# Print the results
print("Highest Elevation City:", highest_elevation_city["name"], "Elevation:", highest_elevation_city["elevation"])
print("Lowest Elevation City:", lowest_elevation_city["name"], "Elevation:", lowest_elevation_city["elevation"])

23/06/01 14:44:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 2986043, Pic de Font Blanca, Pic de Font Blanca, Pic de Font Blanca,Pic du Port, 42.64991, 1.53335, T, PK, AD, , 00, , , , 0, , 2860, Europe/Andorra, 2014-11-05
 Schema: 2986043, Pic de Font Blanca1, Pic de Font Blanca2, Pic de Font Blanca,Pic du Port, 42.64991, 1.53335, T, PK, AD, _c9, 00, _c11, _c12, _c13, 0, _c15, 2860, Europe/Andorra, 2014-11-05
Expected: Pic de Font Blanca1 but found: Pic de Font Blanca
CSV file: file:///workspace/Airport_Pipeline/Geonames_data/allCountries.txt
23/06/01 14:45:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 2986043, Pic de Font Blanca, Pic de Font Blanca, Pic de Font Blanca,Pic du Port, 42.64991, 1.53335, T, PK, AD, , 00, , , , 0, , 2860, Europe/Andorra, 2014-11-05
 Schema: 2986043, Pic de Font Blanca1, Pic de Font Blanca2, Pic de Font Blanca,Pic du Port, 42.64991, 1.53335, T, PK, AD, _c9, 00, _c11, _c12, _c13, 0, _c15, 2860, E

Highest Elevation City: Houston County Elevation: 99
Lowest Elevation City: United States Elevation: None


# 7. Which are the highest and lowest elevated airports, airfields and heliports on the planet?

In [6]:
import pandas as pd

wf = airports[airports["type"].str.contains("closed") == False]
wf = wf[wf["type"].str.contains("seaplane_base") == False]
wf = wf.replace(to_replace=["large_airport", "medium_airport",'small_airport'],value="airport")
airport_types = ['airport', 'heliport']
filtered_df = wf[wf['type'].isin(airport_types)]
filtered_df['elevation_ft'] = filtered_df['elevation_ft'].astype(float)
grouped_df = filtered_df.groupby('type')
max_elevation_df = grouped_df.agg({'elevation_ft': 'max'}).rename(columns={'elevation_ft': 'max_elevation'})
min_elevation_df = grouped_df.agg({'elevation_ft': 'min'}).rename(columns={'elevation_ft': 'min_elevation'})
result_df = pd.concat([max_elevation_df, min_elevation_df], axis=1)
result_df

/tmp/ipykernel_908/2649574585.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['elevation_ft'] = filtered_df['elevation_ft'].astype(float)


,max_elevation,min_elevation
type,,
airport,16200.0,-1266.0
heliport,17372.0,-117.0


In [17]:
spark.stop()